## This is Prod script for loan default prediction

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, time, warnings, gzip, gc, random, math, shap, pickle, optuna, csv, sys, re
from IPython.display import display
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, precision_recall_curve, auc
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score
from sklearn.inspection import permutation_importance
from category_encoders import MEstimateEncoder
from xgboost import XGBClassifier

from google.cloud import bigquery, storage


pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 400)

warnings.filterwarnings("ignore")

app_folder = '/home/jupyter/project_repos/LoanDefault/loans-app'
data_path = '/home/jupyter/projects_data/loans'
project_name = 'My First Project'
project_id = 'quantum-keep-360100'
regionn = 'us-central1'

ml_project_name = 'loans'
model_name = 'XGB'

In [20]:
### target encoding ###
# source: https://www.kaggle.com/code/ryanholbrook/feature-engineering-for-house-prices/notebook

class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=4)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    # To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [ ]:
time0 = time.time()

os.chdir(data_path)
with open(data_path + '/LCLoans_141_800k.pkl', 'rb') as pickled_one:
    df = pickle.load(pickled_one)
    
display(df.head())

In [ ]:
features_tokeep = ['id', 'loan_status',
 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment','issue_d',
 'purpose', 'title', 'initial_list_status', 'application_type',
 'grade', 'sub_grade', 'fico_range_high',
 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'zip_code', 'addr_state',
 'dti',           
 'verification_status', 
 'mo_sin_rcnt_tl', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record',
 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq',
 'num_tl_op_past_12m', 
 'earliest_cr_line', 'inq_last_6mths', 'inq_fi', 'inq_last_12m',
 'open_acc', 'acc_open_past_24mths', 'mort_acc', 'total_acc',
 'avg_cur_bal', 'il_util', 'tot_cur_bal', 
 'revol_bal', 'revol_util', 'max_bal_bc', 'bc_open_to_buy', 'mo_sin_rcnt_rev_tl_op', 'num_actv_rev_tl', 'num_op_rev_tl', 'total_rev_hi_lim',               
 'delinq_2yrs', 'acc_now_delinq', 'delinq_amnt', 'pub_rec', 'pub_rec_bankruptcies',
 'annual_inc_joint', 'dti_joint', 'verification_status_joint',
 'total_bal_ex_mort', 'tot_coll_amt', 'tax_liens', 'percent_bc_gt_75', 'pct_tl_nvr_dlq', 
 'open_rv_12m', 'open_il_12m', 'num_tl_90g_dpd_24m', 'num_tl_30dpd', 'num_tl_120dpd_2m',
 'num_accts_ever_120_pd',
 'recoveries', 'total_rec_prncp', 'total_rec_int']

df = df[features_tokeep]
gc.collect()

recoveries = df[df.loan_status.isin(['Charged Off', 'Default'])][[
    'id', 'loan_status', 'recoveries', 'loan_amnt', 'int_rate', 'total_rec_prncp', 'total_rec_int']]

df.drop(columns = ['recoveries', 'total_rec_prncp', 'total_rec_int'], inplace=True)
# this removes all features, not known to investors ex ante.

df.drop(columns = ['il_util', 'max_bal_bc'], inplace=True)
# these are useful features, which I will preprocess later

df.issue_d = df.issue_d.astype('O')
df.issue_d = pd.to_datetime(df.issue_d, format='%b-%Y')
df['year_issued']=df.issue_d.dt.year
    
#df = df.sample(200000, random_state=1)
df.reset_index(inplace=True, drop=True)
display(df.shape, time.time()-time0, df.head())

In [ ]:
# remove some very rare loan types:

df = df[~df.purpose.isin(['educational', 'renewable_energy', 'wedding'])]
df.purpose = df.purpose.cat.remove_categories(['educational', 'renewable_energy', 'wedding'])

In [ ]:
# clean time features

df.earliest_cr_line = df.earliest_cr_line.astype('O')
df.earliest_cr_line = pd.to_datetime(df.earliest_cr_line, format='%b-%Y')
df['month_issued']=df.issue_d.dt.month
df['year_earliest']=df.issue_d.dt.year
df['years_borrowing'] = (df.issue_d - df.earliest_cr_line)/ np.timedelta64(1, 'Y')
df['pub_rec_pa'] = df.pub_rec/df.years_borrowing
display(df.head())

In [ ]:
# create a target variable

display(df.loan_status.value_counts())
df.target=np.nan
#df.loc[df.loan_status.isin(['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']), 'target']=0
#df.loc[df.loan_status.isin(['Charged Off', 'Late (31-120 days)', 'Does not meet the credit policy. Status:Charged Off', 'Default']), 'target']=1
df.loc[df.loan_status.isin(['Fully Paid']), 'target']=0
df.loc[df.loan_status.isin(['Charged Off', 'Default']), 'target']=1
df=df[df['target'].isin([0,1])]
display(df.shape,df.loan_status.value_counts(), df.count(), sys.getsizeof(df)/1048576)
df.drop(columns='loan_status',inplace=True)

In [ ]:
# add key loan features, scaled by borrower's income:

df.loc[df.annual_inc<1,'annual_inc']=1
df['lti']=df.loan_amnt/df.annual_inc
df['iti']=(df.installment*12)/df.annual_inc
df.loc[df.lti==np.inf, 'lti']=np.nan
df.loc[df.lti>1.5, 'lti']=1.5
df.loc[df.iti==np.inf, 'iti']=np.nan
df.loc[df.iti>1, 'iti']=1
df.loc[df.revol_util>100,'revol_util']=100
df.loc[df.dti>100, 'dti']=100
df.loc[df.dti<0, 'dti']=0

df['revol_balance_income'] = df.revol_bal/df.annual_inc
df['avg_cur_bal_inc'] = df.avg_cur_bal/df.annual_inc
df['tot_cur_bal_inc'] = df.tot_cur_bal/df.annual_inc
df['total_bal_ex_mort_inc'] = df.total_bal_ex_mort/df.annual_inc
df['total_rev_inc'] = df.total_rev_hi_lim/df.annual_inc
df['open_cl_ratio']=df.open_acc/df.total_acc

# add more features

df['zip_code'] = df.zip_code.str.rstrip('xx').astype(int)
df['joint'] = df.dti_joint.notnull().astype(int)
df['emp_length'] = df.emp_length.str.rstrip(' years')
df.loc[df.emp_length=='< 1','emp_length'] = 0
df.loc[df.emp_length=='10+','emp_length'] = 10
df['emp_length'] = df.emp_length.astype(np.float32)
display(df.emp_length.value_counts())
df.amnt_same = (df.loan_amnt == df.funded_amnt_inv).astype(int)
df['low_fico'] = (df.fico_range_high<=659).astype(int)
df.loc[df.home_ownership.isin(['ANY','NONE','OTHER']), 'home_ownership'] = 'OTHER'
df['was_bankrupt'] = (df.pub_rec_bankruptcies>0).astype(int)

df.drop(columns = ['annual_inc_joint', 'dti_joint', 'verification_status_joint', 'earliest_cr_line', 'issue_d'], inplace=True)

df.head()

In [ ]:
# For features like 'time_since some credit event', treat NA as never and fill those values with 100 years equivalent

df.mo_sin_rcnt_tl = df.mo_sin_rcnt_tl.fillna(value=120)
df.num_tl_op_past_12m = df.num_tl_op_past_12m.fillna(value=0)

months_since_col = ['mths_since_last_delinq', 'mths_since_last_major_derog', 
                    'mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq']

for col in months_since_col:
    df[col] = df[col].fillna(value=1200)

#display(df.count())

df.inq_fi = df.inq_fi.fillna(value=0)
df.inq_last_12m = df.inq_last_12m.fillna(value=0)

In [ ]:
cat_features_te = ['sub_grade', 'emp_title', 'purpose', 'title', 'zip_code', 'addr_state', 'grade', 'home_ownership']
cat_features_ohe = ['verification_status', 'initial_list_status', 'application_type']

In [ ]:
#display(df.count())

features_fill_M = ['emp_title', 'title']
features_fill_med = ['dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_util', 
                     'total_acc', 'mort_acc', 'pub_rec_bankruptcies', 'bc_open_to_buy','tot_cur_bal_inc', 'total_rev_inc',
                    'total_bal_ex_mort_inc', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'avg_cur_bal_inc',
                    'mo_sin_rcnt_rev_tl_op', 'num_actv_rev_tl', 'num_op_rev_tl', 'tax_liens']
features_fill_zero = ['open_rv_12m', 'open_il_12m', 'emp_length', 'num_tl_90g_dpd_24m', 
                      'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                     'acc_open_past_24mths', 'tot_coll_amt']

for col in features_fill_zero:
    df[col] = df[col].fillna(value=0)

for col in features_fill_M:
    df[col] = df[col].cat.add_categories(['MISSING']) 
    df[col] = df[col].fillna(value='MISSING')
    
test_size = 0.25
df.reset_index(inplace=True, drop=True)
random.seed(2)
test_index = random.sample(list(df.index), int(test_size*df.shape[0]))
train = df.iloc[list(set(df.index)-set(test_index))]
test = df.iloc[test_index]
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
test00 = test.copy()
train.drop(columns=['id'],inplace=True)
test.drop(columns=['id'],inplace=True)
display(train.shape, test.shape, train.head(3), test.head(3))

for col in features_fill_med:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())
    
train['total_rev_hi_lim'] = train.total_rev_inc.median()*train.annual_inc
test['total_rev_hi_lim'] = train.total_rev_inc.median()*test.annual_inc

train['total_bal_ex_mort'] = train.total_bal_ex_mort_inc.median()*train.annual_inc
test['total_bal_ex_mort'] = train.total_bal_ex_mort_inc.median()*test.annual_inc

train['tot_cur_bal'] = train.tot_cur_bal_inc.median()*train.annual_inc
test['tot_cur_bal'] = train.tot_cur_bal_inc.median()*test.annual_inc

train['avg_cur_bal'] = train.avg_cur_bal_inc.median()*train.annual_inc
test['avg_cur_bal'] = train.avg_cur_bal_inc.median()*test.annual_inc

display(train.count())

In [ ]:
# TE for categorical features

time1 = time.time()
encoder = CrossFoldEncoder(MEstimateEncoder, m=10)
train_encoded = encoder.fit_transform(train, train.target, cols=cat_features_te)
test_encoded = encoder.transform(test)

freq_enc = (train.groupby('title').size()) / len(train)
train['title_fencoded'] = train['title'].apply(lambda x : freq_enc[x])
test['title_fencoded'] = test['title'].apply(lambda x : freq_enc[x])

train.drop(columns=cat_features_te, inplace=True)
test.drop(columns=cat_features_te,  inplace=True)
train = pd.concat([train, train_encoded], axis = 1)
test = pd.concat([test, test_encoded], axis = 1)

display(time.time()-time0, time.time()-time1)
display(train.shape, train.head(), train.count())
train0 = train.copy()
test0 = test.copy()

In [ ]:
X_train = train.copy()
y_train = X_train.pop('target')
X_test = test.copy()
y_test = X_test.pop('target')
display(X_test.head())

### Do OHE for some features ###

feature_transformer = ColumnTransformer([
    ("cat", 
     OneHotEncoder(sparse = False, handle_unknown="ignore", drop='if_binary'), 
     cat_features_ohe)], 
    remainder="passthrough")

print('Number of features before transaformation: ', X_train.shape)
X_train = pd.DataFrame(feature_transformer.fit_transform(X_train), columns=feature_transformer.get_feature_names_out())
X_test = pd.DataFrame(feature_transformer.transform(X_test), columns=feature_transformer.get_feature_names_out())
X_train.columns = X_train.columns.str.replace(r'^cat__', '').str.replace(r'^remainder__', '')
X_test.columns = X_test.columns.str.replace(r'^cat__', '').str.replace(r'^remainder__', '')

print('time to do feature proprocessing: ', time.time()-time1)
print('Number of features after transaformation: ', X_train.shape)

feature_set = ['term', 
               'dti', 
               'acc_open_past_24mths', 
               'lti', 
               'sub_grade_encoded', 
               'emp_title_encoded',
               'home_ownership_encoded']
X_train = X_train[feature_set]
X_test = X_test[feature_set]

X_train.head()
X_test_0 = X_test.copy()

In [ ]:
X_train.columns

In [ ]:
time1 = time.time()

xgbb = XGBClassifier(tree_method = 'gpu_hist',
                    n_estimators = 200,
                    eta = 0.08,
                    max_depth = 5,
                    subsample = 0.8,
                    colsample_bytree = 0.6)

xgbb.fit(X_train, y_train)

precision_t, recall_t, threshold = precision_recall_curve(y_train, xgbb.predict_proba(X_train)[:, 1])
auc_precision_recall_train = auc(recall_t, precision_t)
temp = recall_t[(recall_t>0.095)&(recall_t<0.105)]
temp = temp[int(len(temp)/2)]
indexx = ((np.where(recall_t==temp)))[0][0]
r10prec_train = precision_t[indexx]
precision_t, recall_t, threshold = precision_recall_curve(y_test, xgbb.predict_proba(X_test)[:, 1])
auc_precision_recall_test = auc(recall_t, precision_t)
temp = recall_t[(recall_t>0.095)&(recall_t<0.105)]
temp = temp[int(len(temp)/2)]
indexx = ((np.where(recall_t==temp)))[0][0]
r10prec_test = precision_t[indexx]

display('Train Accuracy: ', accuracy_score(y_train, xgbb.predict(X_train)))
display('F1 score: ', f1_score(y_train, xgbb.predict(X_train)))
display('ROCAUC: ', roc_auc_score(y_train, xgbb.predict(X_train)))
display('PRAUC: ', auc_precision_recall_train)
display('R10P: ', r10prec_train)

# Performance evaluation:
display('Test Accuracy: ', accuracy_score(y_test, xgbb.predict(X_test)))
display('F1 score: ', f1_score(y_test, xgbb.predict(X_test)))
display('ROCAUC: ', roc_auc_score(y_test, xgbb.predict(X_test)))
display('PRAUC: ', auc_precision_recall_test)
display('R10P: ', r10prec_test)
display(time.time()-time1)

fig, ax = plt.subplots()
ax.plot(recall_t, precision_t, color='purple')
ax.set_title('Precision-Recall Curve, test')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_ylim(bottom=0, top=1.02)
plt.show()

xgb_model = xgbb

#### Save model artifact

In [ ]:
model_bucket = 'gs://pmykola-projectsgcp-artifacts/loans'
storage_path = os.path.join(model_bucket, artifact_filename_rf)
blob = storage.blob.Blob.from_string(storage_path, 
                                     client=storage.Client(project=project_id))
blob.upload_from_filename(os.getcwd()+'/'+artifact_filename_rf)

In [ ]:
file = open(artifact_filename_rf, "rb")
trained_model = joblib.load(file)
prediction = trained_model.predict([list(X_test.iloc[0,:])])
print(f'''lm prediction: {prediction}. 
Total time is {time.time()-time0} sec''')

In [55]:
import json

payload = {'instances': [[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.40000000e+03, 6.00000000e+03, 2.40000000e+03,
       6.00000000e+01, 3.59600000e+01, 8.43300018e+01, 4.39000000e+02,
       1.00000000e+01, 1.22520000e+03, 8.72000027e+00, 1.20000000e+02,
       1.20000000e+01, 1.20000000e+01, 1.20000000e+01, 1.20000000e+01,
       1.20000000e+01, 0.00000000e+00, 2.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       1.00000000e+01, 1.39509436e+03, 1.50214629e+04, 2.95600000e+03,
       9.85000000e+01, 5.20100000e+03, 8.00000000e+00, 5.00000000e+00,
       7.00000000e+00, 4.71702002e+03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.95045752e+03,
       0.00000000e+00, 0.00000000e+00, 4.00000000e+01, 9.90000000e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.01100000e+03, 1.20000000e+01,
       2.01100000e+03, 1.00809736e+01, 0.00000000e+00, 1.95886388e-01,
       8.25954974e-02, 2.41266727e-01, 1.13866664e-01, 1.22604167e+00,
       5.67291677e-01, 3.84999990e-01, 2.00000003e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.39409138e-06, 2.55292868e-01,
       2.61714582e-01, 2.84216317e-01, 1.94365262e-01, 1.70245004e-01,
       1.75577218e-01, 2.19904579e-01, 2.25602507e-01], 
                         [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.40000000e+03, 6.00000000e+03, 2.40000000e+03,
       3.60000000e+01, 1.59600000e+01, 8.43300018e+01, 4.39000000e+02,
       1.00000000e+01, 1.22520000e+03, 8.72000027e+00, 1.20000000e+02,
       1.20000000e+01, 1.20000000e+01, 1.20000000e+01, 1.20000000e+01,
       1.20000000e+01, 0.00000000e+00, 2.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       1.00000000e+01, 1.39509436e+03, 1.50214629e+04, 2.95600000e+03,
       9.85000000e+01, 5.20100000e+03, 8.00000000e+00, 5.00000000e+00,
       7.00000000e+00, 4.71702002e+03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.95045752e+03,
       0.00000000e+00, 0.00000000e+00, 4.00000000e+01, 9.90000000e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.01100000e+03, 1.20000000e+01,
       2.01100000e+03, 1.00809736e+01, 0.00000000e+00, 1.95886388e-01,
       8.25954974e-02, 2.41266727e-01, 1.13866664e-01, 1.22604167e+00,
       5.67291677e-01, 3.84999990e-01, 2.00000003e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.39409138e-06, 2.55292868e-01,
       2.61714582e-01, 2.84216317e-01, 1.94365262e-01, 1.70245004e-01,
       1.75577218e-01, 2.19904579e-01, 2.25602507e-01]]}

# Parse JSON
with open('request.json', 'w') as outfile:
    json.dump(payload, outfile)

!gcloud ai endpoints predict $endpoint_id \
  --region=$regionn \
  --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[0.4651663899421692, 0.3640084266662598]


To take a quick anonymous survey, run:
  $ gcloud survey

